# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart workflow
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Setup
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))

from gu_toolkit import *


## 1) Quickstart: Minimal interactive figure

This pattern covers most exploratory notebook work:
- create a figure,
- define one or more plots,
- bind parameters by name,
- render.


In [ ]:
x, a, b = sp.symbols("x a b")
fig = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=500)

with fig:
    plot(x, a*sp.sin(b*x), parameters=[a, b], id="wave", label="a*sin(bx)")
    parameter(a, value=1.0, min=-2.0, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.2, max=4.0, step=0.1)
    set_title("Quickstart: Interactive sine")
    render()


## 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
# Compact utility snippets
with fig:
    print("x-range:", get_x_range())
    print("y-range:", get_y_range())
    print("param snapshot:", params.snapshot())


## 3) Parameter patterns you will reuse often

### A) Explicit slider bounds and defaults
Useful for reproducible demos.


In [ ]:
x, amp, freq, phase = sp.symbols("x amp freq phase")
fig2 = Figure(x_range=(-10, 10), y_range=(-2, 2), sampling_points=600)

with fig2:
    plot(x, amp*sp.cos(freq*x + phase), parameters=[amp, freq, phase], id="cosine")
    parameter(amp, value=1.0, min=0.0, max=2.0, step=0.05)
    parameter(freq, value=1.0, min=0.1, max=3.0, step=0.05)
    parameter(phase, value=0.0, min=-pi, max=pi, step=0.1)
    set_title("Cosine with amplitude/frequency/phase controls")
    render()


### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
f = numpify(expr, vars=["x", DYNAMIC_PARAMETER, DYNAMIC_PARAMETER, DYNAMIC_PARAMETER])

x, A, k, phi = sp.symbols("x A k phi")
fig3 = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=700)
with fig3:
    plot(x, A*sp.sin(k*x + phi), parameters=[A, k, phi], id="symbolic-wave")
    parameter(A, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(k, value=1.0, min=0.2, max=4.0, step=0.1)
    parameter(phi, value=0.0, min=-pi, max=pi, step=0.1)
    set_title("Symbolic expression with dynamic parameters")
    render()


## 4) Debugging and introspection helpers

These checks are useful when a notebook cell is not behaving as expected.


In [ ]:
with fig3:
    print("Current params:", params.snapshot())
print("Figure has", len(fig3.plots), "plot(s)")
print("Plot ids:", list(fig3.plots.keys()))

plot_obj = fig3.plots["symbolic-wave"]
print("Plot label:", plot_obj.label)
print("Has cached samples:", plot_obj.cached_samples is not None)


## 5) Cool applications

### Application 1: Harmonic synthesizer
A compact additive synthesis visualization.


In [ ]:
x, a1, a2, a3 = sp.symbols("x a1 a2 a3")
fig4 = Figure(x_range=(-2*pi, 2*pi), y_range=(-4, 4), sampling_points=900)

with fig4:
    plot(x, a1*sp.sin(x) + a2*sp.sin(2*x) + a3*sp.sin(3*x), parameters=[a1, a2, a3], id="harmonic")
    parameter(a1, value=1.0, min=-2.0, max=2.0, step=0.1)
    parameter(a2, value=0.0, min=-2.0, max=2.0, step=0.1)
    parameter(a3, value=0.0, min=-2.0, max=2.0, step=0.1)
    set_title("Harmonic synthesizer")
    render()


### Application 2: Gaussian mixture explorer
Great for intuition about peak width, center, and mixture weights.


In [ ]:
x, w1, w2, m1, m2, s1, s2 = sp.symbols("x w1 w2 m1 m2 s1 s2")
expr = w1*sp.exp(-sp.Rational(1, 2)*((x - m1)/s1)**2) + w2*sp.exp(-sp.Rational(1, 2)*((x - m2)/s2)**2)

fig5 = Figure(x_range=(-8, 8), y_range=(0, 2), sampling_points=800)
with fig5:
    plot(x, expr, parameters=[w1, w2, m1, m2, s1, s2], id="mixture")
    parameter(w1, value=1.0, min=0.0, max=1.5, step=0.05)
    parameter(w2, value=0.8, min=0.0, max=1.5, step=0.05)
    parameter(m1, value=-1.5, min=-5.0, max=5.0, step=0.1)
    parameter(m2, value=1.5, min=-5.0, max=5.0, step=0.1)
    parameter(s1, value=1.0, min=0.2, max=3.0, step=0.1)
    parameter(s2, value=0.8, min=0.2, max=3.0, step=0.1)
    set_title("Gaussian mixture explorer")
    render()


## 6) Practical checklist

When prototyping in notebooks:
- Start with one plot and one parameter.
- Confirm `params.snapshot()` updates while sliders move.
- Increase `sampling_points` only when needed.
- Use clear `id=` values for plots you update repeatedly.

This notebook is intended as a durable daily-use reference.
